In [1]:
import numpy as np
import pandas as pd
import re
import whois
import time
from tqdm.auto import tqdm

In [2]:
import sys 
sys.path.append('/Users/keremkosif/Desktop/playground/url_features/scripts')

In [3]:
    from feature_functions import FeatureFunctions
    from feature_extractor import FeatureExtractor
    from external_functions import ExternalFunctions
    from url_parser import UrlParser

    from rank import Rank

    ranker = Rank()
    functions = FeatureFunctions()
    future_e = FeatureExtractor()
    ef = ExternalFunctions()
    up = UrlParser()

In [4]:
hints = ["porn", "secure", "update", "login", "signin", "bank", "confirm", "verify"]

In [5]:
df_columns = ['has_ip', 'url_len', 'is_short', 'nb_semico', 'nb_dslash', 'nb_slash',
              'nb_colon', 'nb_tilde', 'abn_sd', 'nb_sd', 'nb_dot', 'nb_at', 'nb_and',
              'nb_equal', 'nb_excl', 'nb_uscore', 'nb_hyphens', 'd_ratio', 'domain_s',
              'domain_e', 'count_https', 'nb_hint', 'Url', 'Google PageRank', 'cPR Score',
              'Domain Authority', 'Page Authority', 'Trust Flow', 'Trust Metric',
              'Citation Flow', 'Domain Validity', 'Global Rank', 'Alexa USA Rank',
              'Alexa Reach Rank', 'Spam Score', 'External Backlinks', 'Referring Domains',
              'EDU Backlinks', 'EDU Domains', 'GOV Backlinks', 'GOV Domains',
              'PR Quality', 'Domain Age', 'HTTP Response Codes:', 'Google Directory listed',
              'DMOZ.org listed', 'Canonical URL',
              'Root IP', 'Title', 'Topic:', 'Topic Value', 'Indexed URLs', 'Crawled Flag']

In [59]:
url_columns = ['has_ip', 'url_len', 'is_short', 'nb_semico', 'nb_dslash', 'nb_slash',
              'nb_colon', 'nb_tilde', 'abn_sd', 'nb_sd', 'nb_dot', 'nb_at', 'nb_and',
              'nb_equal', 'nb_excl', 'nb_uscore', 'nb_hyphens', 'd_ratio', 'domain_s',
              'domain_e', 'count_https', 'nb_hint', "Url"]

In [6]:
rank_columns = ['Google PageRank', 'cPR Score', 'Domain Authority', 'Page Authority', 'Trust Flow', 'Trust Metric',
                'Citation Flow', 'Domain Validity', 'Global Rank', 'Alexa USA Rank', 'Alexa Reach Rank', 'Spam Score',
                'External Backlinks', 'Referring Domains', 'EDU Backlinks', 'EDU Domains', 'GOV Backlinks', 'GOV Domains',
                'PR Quality', 'Domain Age', 'HTTP Response Codes:', 'Google Directory listed', 'DMOZ.org listed', 'Canonical URL',
                'Root IP', 'Title', 'Topic:', 'Topic Value', 'Indexed URLs', 'Crawled Flag']

In [7]:
full_functions = [functions.having_ip_address, functions.url_length, functions.shortening_service,
                  functions.count_semicolumn,functions.count_double_slash, functions.count_slash,
                  functions.count_colon ,functions.count_tilde,
                  functions.abnormal_subdomain, functions.count_subdomain, functions.count_dots,
                  functions.count_at,
                  functions.count_and, functions.count_equal, functions.count_exclamation,
                  functions.count_underscore, functions.count_hyphens,]

In [8]:
base_functions = [functions.ratio_digits, ef.creation_date_len, ef.expiration_date_len,]
                  #ef.whois_registered_domain, ef.registrant_country, ef.registrar]

In [9]:
df = pd.read_csv("output/jotform_test.csv")
urls = df["url"]
#urls = ["http://www.w3.org/1999/xhtml", "https://ci6.googleusercontent.com/proxy/Xfn_CiZC9oS3brrV1tfKidMDtE_DzzmoaetAsAxYoG1mnHCb4OfAyRle1JmoGdEfy3MulU1UygdYfi-fRQhLKY2GDYWps9u5v9xS9Lc=s0-d-e1-ft",
#        "https://cheaptrade.pt/us/ups/", "https://bit.ly/3M0PkNm"]

In [79]:
df

,url,label
0,http://www.w3.org/1999/xhtml,1
1,https://ci6.googleusercontent.com/proxy/Xfn_Ci...,1
2,https://cheaptrade.pt/us/ups/,1
3,https://bit.ly/3M0PkNm,1
4,https://royaltextileofficial.com/new,1
...,...,...
1453,https://www.qube-learning.co.uk/hgv-bootcamp/,0
1454,https://www.posterpresentations.com/research/g...,0
1455,https://www.solinst.com,0
1456,https://sid.iisc.ac.in/careers/,0


In [10]:
def url_features():
    pbar = tqdm(total=len(urls))
    df_list = []
    
    for i in range(len(urls)):
        url = urls[i]
        p_url = up.return_url_parts(url)
        url_row = []

        ### FUNCTION GROUP ONE FULL URL ###
        for function in full_functions:
            ex_f = lambda arg : function(arg)
            result = ex_f(url)
            url_row.append(result)

        ### FUNCTION GROUP TWO DOMAIN ###
        for function in base_functions:
            ex_f = lambda arg : function(arg)
            try:
                result = ex_f(p_url["domain"])
            except:
                result = "unknown"
            url_row.append(result)

        ### OTHER FUNCTIONS ###
        url_row.append(functions.count_http_token(p_url["path"]))
        url_row.append(functions.phish_hints(url, hints))
        url_row.append(functions.https_token(p_url["protocol"]))

        df_list.append(url_row)
        pbar.update(1)
        
    return df_list

url_f = url_features()

  0%|          | 0/1458 [00:00<?, ?it/s]

In [11]:
df["url"][629]

'http://help'

In [12]:
def rank_features():
    to_begin = 600
    input_size = len(urls)
    pbar = tqdm(total=input_size-to_begin)
    df_list = []

    i = to_begin
    trial = 0
    while i < input_size:
        url = df["url"]
        s_url = df["url"][i]
        try:
            domain = ranker.return_url_parts(s_url)["domain"]
        except:
            # This exeption occurs when urls are broken.
            df_list.append([])
            i += 1
            pbar.update(1)
            continue
        try:
            scraps = ranker.rank_datas(domain, trial)
        except:
            i += 1
            pbar.update(1)
            continue

        if scraps == -1:
            trial = 1
            continue
            
        df_list.append(scraps)
        i += 1
        pbar.update(1)
        trial = 0 
        # 592
        
    pbar.close()
    return df_list

rank_f = rank_features()

  0%|          | 0/858 [00:00<?, ?it/s]

In [33]:
c_rank = rank_f.copy()

In [37]:
tpc = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]

In [38]:
for i in range(len(c_rank)):
    if c_rank[i] == []:
        c_rank[i] == tpc

In [39]:
df_1 = pd.read_csv("output/0-600.csv")

In [41]:
df_5 = pd.DataFrame(c_rank, columns=rank_columns)

In [54]:
df_son = pd.concat([df_1, df_5], axis=0)

In [60]:
df_ur = pd.DataFrame(url_f, columns=url_columns)

In [64]:
del df_ur["Url"]

In [69]:
#df_final = pd.concat([df_son, df_ur], axis=1)

In [67]:
df_son.shape

(1458, 30)

In [68]:
df_ur.shape

(1458, 22)

In [ ]:
#df_6 = pd.DataFrame(url_f, columns=rank_columns)

In [73]:
df_son.to_numpy()

array([['9/10', '9.2/10', '94', ..., '94', '213,213,908', 'False'],
       ['8/10', '8.6/10', '97', ..., '17', '5,341', 'False'],
       ['2/10', '2.1/10', '11', ..., '0', '1,788,596', 'False'],
       ...,
       ['4/10', '4.3/10', '37', ..., '15', '1,126', 'False'],
       ['5/10', '5.8/10', '63', ..., '11', '1,526', 'False'],
       ['8/10', '9.0/10', '94', ..., '60', '345,845,742', 'False']],
      dtype=object)

In [74]:
arr1 = np.array(url_f)
arr2 = df_son.to_numpy()
df_array = np.concatenate((arr1,arr2),axis=1)

In [77]:
final_df = pd.DataFrame(df_array, columns=df_columns)

In [78]:
final_df.to_csv("output/features.csv")

In [55]:
#arr1 = np.array(url_f)
#arr2 = np.array(rank_f)
#df_array = np.concatenate((arr1,arr2),axis=1)

In [ ]:
#df = pd.DataFrame(df_array, columns=df_columns)

In [ ]:
df = df.drop(['Topic:', "Title", "Global Rank", "Alexa USA Rank", "Alexa Reach Rank",
              "Domain Age", "HTTP Response Codes:", "Canonical URL", "Root IP",
              "Trust Flow", "cPR Score", "Domain Authority", "Page Authority","Referring Domains",
              'GOV Domains', 'GOV Backlinks', 'EDU Domains'], axis=1)

In [56]:
df_son.to_csv("output/features.csv", index=False)